In [1]:
import numpy as np

class GaussianNB:
    """
    Gaussian Naive Bayes for continuous features.
    """

    def fit(self, X, y):
        """
        Fit the Gaussian Naive Bayes classifier from training set.

        Parameters:
        -----------
        X : numpy.ndarray
            Training features (n_samples, n_features).
        y : numpy.ndarray
            Target labels (n_samples,). Should be integers (0, 1, ..., n_classes-1).
        """
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        n_features = X.shape[1]
        self.class_priors = np.zeros(n_classes)
        self.mean = np.zeros((n_classes, n_features))
        self.var = np.zeros((n_classes, n_features))
        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.class_priors[idx] = X_c.shape[0] / X.shape[0]
            self.mean[idx, :] = np.mean(X_c, axis=0)
            self.var[idx, :] = np.var(X_c, axis=0) + 1e-9  # avoid zero variance

    def _pdf(self, class_idx, x):
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp(- (x - mean) ** 2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator

    def predict(self, X):
        posteriors = []
        for x in X:
            class_posteriors = []
            for idx, c in enumerate(self.classes):
                prior = np.log(self.class_priors[idx])
                likelihood = np.sum(np.log(self._pdf(idx, x)))
                class_posteriors.append(prior + likelihood)
            posteriors.append(self.classes[np.argmax(class_posteriors)])
        return np.array(posteriors)

class MultinomialNB:
    """
    Multinomial Naive Bayes for count-based features (e.g., text).
    """

    def fit(self, X, y):
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        n_features = X.shape[1]
        self.class_priors = np.zeros(n_classes)
        self.feature_probs = np.zeros((n_classes, n_features))
        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.class_priors[idx] = X_c.shape[0] / X.shape[0]
            # Laplace smoothing
            self.feature_probs[idx, :] = (np.sum(X_c, axis=0) + 1) / (np.sum(X_c) + n_features)

    def predict(self, X):
        posteriors = []
        for x in X:
            class_posteriors = []
            for idx, c in enumerate(self.classes):
                prior = np.log(self.class_priors[idx])
                likelihood = np.sum(x * np.log(self.feature_probs[idx, :]))
                class_posteriors.append(prior + likelihood)
            posteriors.append(self.classes[np.argmax(class_posteriors)])
        return np.array(posteriors)

class BernoulliNB:
    """
    Bernoulli Naive Bayes for binary features (0 or 1).
    """

    def fit(self, X, y):
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        n_features = X.shape[1]
        self.class_priors = np.zeros(n_classes)
        self.feature_probs = np.zeros((n_classes, n_features))
        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.class_priors[idx] = X_c.shape[0] / X.shape[0]
            # Laplace smoothing
            self.feature_probs[idx, :] = (np.sum(X_c, axis=0) + 1) / (X_c.shape[0] + 2)

    def predict(self, X):
        posteriors = []
        for x in X:
            class_posteriors = []
            for idx, c in enumerate(self.classes):
                prior = np.log(self.class_priors[idx])
                prob = self.feature_probs[idx, :]
                # log(p^x * (1-p)^(1-x)) = x*log(p) + (1-x)*log(1-p)
                likelihood = np.sum(x * np.log(prob) + (1-x) * np.log(1-prob))
                class_posteriors.append(prior + likelihood)
            posteriors.append(self.classes[np.argmax(class_posteriors)])
        return np.array(posteriors)

# Example usage:
if __name__ == "__main__":
    # GaussianNB: Iris-style data
    print("GaussianNB Example")
    Xg = np.array([[5.1, 3.5], [4.9, 3.0], [6.2, 3.4], [5.9, 3.0]])
    yg = np.array([0, 0, 1, 1])
    model_g = GaussianNB()
    model_g.fit(Xg, yg)
    print("Predicted:", model_g.predict(np.array([[6.0, 3.2], [5.0, 3.6]])))

    # MultinomialNB: Text count
    print("MultinomialNB Example")
    Xm = np.array([[2, 1, 0], [1, 2, 1], [2, 0, 2], [0, 1, 2]])
    ym = np.array([0, 0, 1, 1])
    model_m = MultinomialNB()
    model_m.fit(Xm, ym)
    print("Predicted:", model_m.predict(np.array([[1, 1, 1], [2, 0, 1]])))

    # BernoulliNB: Binary features
    print("BernoulliNB Example")
    Xb = np.array([[1, 0, 1], [0, 1, 0], [1, 1, 1], [0, 0, 1]])
    yb = np.array([0, 0, 1, 1])
    model_b = BernoulliNB()
    model_b.fit(Xb, yb)
    print("Predicted:", model_b.predict(np.array([[1, 0, 1], [0, 1, 0]])))

GaussianNB Example
Predicted: [1 0]
MultinomialNB Example
Predicted: [0 1]
BernoulliNB Example
Predicted: [1 0]
